In [1]:
import bugzilla
import requests
import time
from datetime import datetime

In [2]:
pagure_owner_alias = requests.get("https://src.fedoraproject.org/extras/pagure_owner_alias.json").json()["rpms"]

In [3]:
orphaned = [p for p in pagure_owner_alias if "orphan" in pagure_owner_alias[p]]
orphaned[:10]

['7kaa',
 '8Kingdoms',
 '915resolution',
 '99soft-oss-parent',
 'AcetoneISO2',
 'Ajaxterm',
 'CableSwig',
 'Canna',
 'CastPodder',
 'CodeAnalyst-gui']

In [4]:
bzapi = bugzilla.Bugzilla("bugzilla.redhat.com")

In [5]:
if not bzapi.logged_in:
    bzapi.interactive_login()

In [6]:
TRACKER = 1700317

query = bzapi.build_query(product="Fedora", include_fields=["flags", "assigned_to", "component", "creation_time"])
query["blocks"] = TRACKER
query["status"] = "NEW"
bugzillas = bzapi.query(query)
bugzillas[:10]

[<Bug #1701227 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f93986f0390>,
 <Bug #1714212 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f93986f0a50>,
 <Bug #1716412 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f939960ba10>,
 <Bug #1716488 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f939960bb50>,
 <Bug #1716525 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f939960b510>,
 <Bug #1716546 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f9399608fd0>,
 <Bug #1729908 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f9399608e50>,
 <Bug #1734875 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f9399608f50>,
 <Bug #1734882 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f9399608cd0>,
 <Bug #1734884 on https://bugzilla.redhat.com/xmlrpc.cgi at 0x7f9399608e90>]

In [7]:
RELENG = "releng@fedoraproject.org"
ORPHAN = "extras-orphan@fedoraproject.org"

In [8]:
pkgs_to_orphan = []
bugs_to_reassign = []

In [9]:
for bug in bugzillas:
    if bug.component in orphaned:
        if bug.assigned_to != ORPHAN:
            bugs_to_reassign.append(bug.id)
        continue
    needinfos = bug.get_flags("needinfo")
    if not needinfos:
        continue
    for needinfo in needinfos:
        if needinfo['requestee'] == bug.assigned_to:
            break
    else:
        # no needinfo on assignee
        continue
    bug_state = 0
    last_time = datetime.fromtimestamp(time.mktime(bug.creation_time.timetuple()))
    for comment in bug.getcomments():
        if comment["creator"] != RELENG:
            continue
        if not comment["text"].startswith("Dear Maintainer,"):
            continue
        comment_time = datetime.fromtimestamp(time.mktime(comment["creation_time"].timetuple()))
        delta = comment_time - last_time
        if bug_state == 0 and delta.days >= 7-1:
            last_time = comment_time
            bug_state = 1
        elif bug_state == 1 and delta.days >= 3*7-1:
            last_time = comment_time
            bug_state = 2
    delta = datetime.now() - last_time
    if bug_state == 2 and delta.days >= 4*7-1:
        to_orphan = f"rpms/{bug.component}"
        pkgs_to_orphan.append(to_orphan)
        print(to_orphan)
        bugs_to_reassign.append(bug.id)

In [10]:
print("https://bugzilla.redhat.com/buglist.cgi?bug_id=" + ",".join(str(b) for b in bugs_to_reassign))

https://bugzilla.redhat.com/buglist.cgi?bug_id=1701227,1716525,1735058,1735115,1735855,1736046,1736474,1736950,1736988,1736989,1739634
